<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Phase1_LOB_pencil_sketch_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Phase 1 — Pencil Sketch (Single Tick, 4 Agents) ✅

This Colab notebook lets you **log events for 4 agents (a/b/c/d)** at a **single price level** and **auto‑compute**:

- **H**: Hot‑potato count (Sell@Ask → Buy@Ask by the same agent within `τ` seconds, same qty)  
- **QueuePosOnExit** distribution (you fill it manually when an agent exits)  
- **Round‑trip P&L** (≈ 0 by construction for same‑price, same‑qty hot‑potato)

### How to use
1. **Run all** cells.
2. Set **Config** (`p_b`, `p_a`, `tau_sec`).  
3. Confirm **Agents** (A/B/C/D roles, `I`, `S`) or edit if needed.  
4. Use the **Event Logger** widget to add events in time order.  
5. See **Derived Metrics** update live.  
6. You can **download/upload** Events/Agents as CSV.

> Scope: *single tick* (best bid/ask only), *no* multi‑price dynamics. This is a concept lock‑in tool before heavy coding.


In [1]:

import pandas as pd
import numpy as np
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

# --- Config ---
p_b = 9.10       # Best Bid
p_a = 9.11       # Best Ask
tau_sec = 5      # τ window in seconds for hot-potato

print(f"Config → Bid={p_b}, Ask={p_a}, τ={tau_sec}s")


Config → Bid=9.1, Ask=9.11, τ=5s


In [2]:

agents_cols = ["Agent","Role_t0","EntryPrice","I","S","Notes"]
agents_df = pd.DataFrame([
    ["a","a.0 MarketBuy",   p_a,  1000, -p_a*1000, "Aggressive buy crossing to ask"],
    ["b","b.0 LimitSell",   p_a, -1000,  p_a*1000, "Passive sell at ask (maker)"],
    ["c","c.0 MarketSell",  p_b, -1000,  p_b*1000, "Aggressive sell hitting bid"],
    ["d","d.0 LimitBuy",    p_b,  1000, -p_b*1000, "Passive buy at bid (maker)"],
], columns=agents_cols)

display(HTML("<h3>Agents_t0</h3>"))
display(agents_df)


,Agent,Role_t0,EntryPrice,I,S,Notes
0,a,a.0 MarketBuy,9.11,1000,-9110.0,Aggressive buy crossing to ask
1,b,b.0 LimitSell,9.11,-1000,9110.0,Passive sell at ask (maker)
2,c,c.0 MarketSell,9.10,-1000,9100.0,Aggressive sell hitting bid
3,d,d.0 LimitBuy,9.10,1000,-9100.0,Passive buy at bid (maker)


In [3]:

event_cols = [
    "t_sec","agent","action","side","price","qty",
    "P_or_A","Exit?","QueuePosOnExit",
    "isSellAtAsk","isBuyAtAsk","HP_matchRow","HP_flag","RoundTrip_PnL"
]
events_df = pd.DataFrame(columns=event_cols)

def compute_derived(df, p_a, tau_sec):
    if df.empty:
        return df.copy(), 0, np.nan, 0.0

    out = df.copy()
    out["isSellAtAsk"] = (out["action"].eq("MarketSell")) & (out["price"].astype(float) == float(p_a))
    out["isBuyAtAsk"]  = (out["action"].eq("MarketBuy"))  & (out["price"].astype(float) == float(p_a))

    # For each row that's Sell@Ask, find the earliest Buy@Ask by same agent within tau and same qty
    out["HP_matchRow"] = ""
    out["HP_flag"] = 0
    out["RoundTrip_PnL"] = ""

    for i,row in out[out["isSellAtAsk"]].iterrows():
        agent_i = row["agent"]
        t0 = float(row["t_sec"])
        q0 = float(row["qty"])
        # candidates
        cand = out[
            (out.index >= i) &
            (out["agent"] == agent_i) &
            (out["isBuyAtAsk"]) &
            (out["t_sec"].astype(float) >= t0) &
            (out["t_sec"].astype(float) <= t0 + float(tau_sec)) &
            (out["qty"].astype(float) == q0)
        ]
        if len(cand)>0:
            j = cand.index[0]
            out.at[i,"HP_matchRow"] = int(j)
            out.at[i,"HP_flag"] = 1
            out.at[i,"RoundTrip_PnL"] = 0.0  # same price/qty → 0 by construction

    # Metrics
    H = int(out["HP_flag"].sum())
    qpos_vals = pd.to_numeric(out.loc[out["Exit?"].astype(str).str.upper().eq("Y"), "QueuePosOnExit"], errors="coerce")
    avg_qpos = float(qpos_vals.mean()) if qpos_vals.notna().any() else np.nan
    total_rtpnl = pd.to_numeric(out["RoundTrip_PnL"], errors="coerce").fillna(0).sum()
    return out, H, avg_qpos, total_rtpnl

events_df, H, avg_qpos, total_rtpnl = compute_derived(events_df, p_a, tau_sec)

display(HTML("<h3>Events (start empty)</h3>"))
display(events_df)
display(HTML(f"<h4>Derived → H={H}, Avg QueuePosOnExit={avg_qpos}, Total RoundTrip P&L={total_rtpnl}</h4>"))


,t_sec,agent,action,side,price,qty,P_or_A,Exit?,QueuePosOnExit,isSellAtAsk,isBuyAtAsk,HP_matchRow,HP_flag,RoundTrip_PnL


In [4]:

# Dropdowns and inputs
agent_dd = widgets.Dropdown(options=["a","b","c","d"], description="Agent:")
action_dd = widgets.Dropdown(options=["LimitAddBid","LimitAddAsk","MarketBuy","MarketSell","Cancel","Execute"], description="Action:")
side_dd = widgets.Dropdown(options=["bid","ask",""], description="Side:")
price_tb = widgets.FloatText(value=p_a, description="Price:")
qty_tb = widgets.IntText(value=1000, description="Qty:")
porA_dd = widgets.Dropdown(options=["P","A",""], description="P/A:")
exit_dd = widgets.Dropdown(options=["","Y","N"], description="Exit?:")
qpos_tb = widgets.IntText(value=None, description="QueuePos:")
tsec_tb = widgets.FloatText(value=0.0, description="t_sec:")

add_btn = widgets.Button(description="Add Event", button_style="primary")
reset_btn = widgets.Button(description="Reset Events", button_style="warning")

out_area = widgets.Output()

def add_event(_):
    global events_df, H, avg_qpos, total_rtpnl
    row = {
        "t_sec": tsec_tb.value,
        "agent": agent_dd.value,
        "action": action_dd.value,
        "side": side_dd.value,
        "price": price_tb.value,
        "qty": qty_tb.value,
        "P_or_A": porA_dd.value,
        "Exit?": exit_dd.value,
        "QueuePosOnExit": qpos_tb.value,
        "isSellAtAsk": None,
        "isBuyAtAsk": None,
        "HP_matchRow": "",
        "HP_flag": 0,
        "RoundTrip_PnL": ""
    }
    events_df = pd.concat([events_df, pd.DataFrame([row])], ignore_index=True)
    # recompute
    events_df, H, avg_qpos, total_rtpnl = compute_derived(events_df, p_a, tau_sec)
    with out_area:
        clear_output(wait=True)
        display(HTML("<h3>Events</h3>"))
        display(events_df)
        display(HTML(f"<h4>Derived → H={H}, Avg QueuePosOnExit={avg_qpos}, Total RoundTrip P&L={total_rtpnl}</h4>"))

def reset_events(_):
    global events_df, H, avg_qpos, total_rtpnl
    events_df = pd.DataFrame(columns=[
        "t_sec","agent","action","side","price","qty",
        "P_or_A","Exit?","QueuePosOnExit",
        "isSellAtAsk","isBuyAtAsk","HP_matchRow","HP_flag","RoundTrip_PnL"
    ])
    events_df, H, avg_qpos, total_rtpnl = compute_derived(events_df, p_a, tau_sec)
    with out_area:
        clear_output(wait=True)
        display(HTML("<h3>Events (reset)</h3>"))
        display(events_df)
        display(HTML(f"<h4>Derived → H={H}, Avg QueuePosOnExit={avg_qpos}, Total RoundTrip P&L={total_rtpnl}</h4>"))

add_btn.on_click(add_event)
reset_btn.on_click(reset_events)

ui = widgets.VBox([
    widgets.HBox([tsec_tb, agent_dd, action_dd]),
    widgets.HBox([side_dd, price_tb, qty_tb]),
    widgets.HBox([porA_dd, exit_dd, qpos_tb]),
    widgets.HBox([add_btn, reset_btn]),
    out_area
])

display(HTML("<h3>Event Logger</h3>"))
display(ui)


In [5]:

import io
from google.colab import files

def download_csv(df, fname):
    df.to_csv(fname, index=False)
    files.download(fname)

download_events_btn = widgets.Button(description="Download Events CSV")
download_agents_btn = widgets.Button(description="Download Agents CSV")

def _dl_events(_):
    download_csv(events_df, "events_phase1.csv")

def _dl_agents(_):
    download_csv(agents_df, "agents_t0_phase1.csv")

upload_events = widgets.FileUpload(accept=".csv", multiple=False, description="Upload Events CSV")
upload_agents = widgets.FileUpload(accept=".csv", multiple=False, description="Upload Agents CSV")

def _ul_events(change):
    global events_df, H, avg_qpos, total_rtpnl
    if upload_events.value:
        content = list(upload_events.value.values())[0]['content']
        events_df = pd.read_csv(io.BytesIO(content))
        events_df, H, avg_qpos, total_rtpnl = compute_derived(events_df, p_a, tau_sec)
        with out_area:
            clear_output(wait=True)
            display(HTML("<h3>Events (uploaded)</h3>"))
            display(events_df)
            display(HTML(f"<h4>Derived → H={H}, Avg QueuePosOnExit={avg_qpos}, Total RoundTrip P&L={total_rtpnl}</h4>"))

def _ul_agents(change):
    global agents_df
    if upload_agents.value:
        content = list(upload_agents.value.values())[0]['content']
        agents_df = pd.read_csv(io.BytesIO(content))
        with out_area:
            display(HTML("<h3>Agents (uploaded)</h3>"))
            display(agents_df)

download_events_btn.on_click(_dl_events)
download_agents_btn.on_click(_dl_agents)
upload_events.observe(_ul_events, names='value')
upload_agents.observe(_ul_agents, names='value')

display(HTML("<h3>Save / Load</h3>"))
display(widgets.HBox([download_events_btn, download_agents_btn]))
display(widgets.HBox([upload_events, upload_agents]))
